In [1]:
companies = ['GE', 'F', 'AMD', 'FIT', 'GPRO', 'SNAP', 'CRON', 'CHK', 'SIRI', 'XXII', 'ZNGA', 'S', 'GRPN', 'RAD',
             'PLUG', 'HMNY', 'NVAX', 'INSY', 'UAA', 'APRN', 'GLUU', 'TRXC', 'AUY', 'BOTZ', 'GERN', 'BILI', 'JCP',
            'CARA', 'VSLR', 'P', 'TEVA', 'RUN', 'UA', 'NOK', 'SPWR', 'CY']

In [ ]:
actions = []
for ticker in companies:
    current = data.DataReader(ticker, 'iex', dt.datetime.now() - dt.timedelta(days=3), dt.datetime.now())
    
    try:
        last_close = current['close'].values[0]
    except:
        continue
        
    # make predictions
    try:
        max_move, expected_move, min_move = make_predictions(ticker, 30, False)
    except ValueError as e:
        continue

    # calculate partition for gain and loss
    spread = abs(max_move - min_move)
    if max_move < 0:
        gain_partition = 0
        loss_partition = 1
    elif min_move > 0:
        gain_partition = 1
        loss_partition = 0
    else:
        gain_partition = max_move / spread
        loss_partition = abs(min_move) / spread
        
    # view pie chart comparing gain and loss
#     fig2, ax2 = plt.subplots()
#     ax2.pie([gain_partition, loss_partition], labels=['gain', 'loss'])
#     ax2.set_title(ticker + " Gain vs Loss")
    
    # determine potential trades
    if expected_move > 5 and gain_partition > 67:
        actions.append({ "ticker": ticker, "action": "STRONG CALL" })
        
    elif expected_move > 0 and gain_partition > 50:
        actions.append({ "ticker": ticker, "action": "WEAK CALL" })
        
    elif expected_move < 0 and loss_partition > 50:
        actions.append({ "ticker": ticker, "action": "WEAK PUT" })
    
    elif expected_move < -5 and loss_partition > 67:
        actions.append({ "ticker": ticker, "action": "STRONG PUT" })
    
pd.DataFrame(actions).to_csv("trades.csv", index=False)